## Imports

In [ ]:
from sumedipose_utils import *
import numpy as np
import json
import pandas as pd

## Set variables

In [ ]:
subject = 'S1' # You can choose any subject
action = 'A1' # You can choose any action
speed = 'D1' # You can choose any speed
main = 1 # Choose between 1 and 6
pair1 = 2 # Choose 2 if main is 1, choose 4 if main is 6
pair2 = 3 # Choose 3 if main is 1, choose 5 if main is 6
root_path = 'root/path/to/data/' # change this to the path where the data is stored
path_3d = f'{root_path}WCS/{subject}/{subject}{action}{speed}.json'
cal_path = f'{root_path}calibration/external_calibration.csv' # path to calibration file
wand_path = f'{root_path}markers_points.json' # path to wand points file
cal_size = 60 # size of the calibration grid
index = 0 # Choose any index you want to visualize or use

## Map subjects to camera matrices
The same matrices are used for all the subjects captured within one day and the day is labbled by the first subject captured. This mapping maps all the subjects within the day to the first subject to identify the correct parameters to use.

In [ ]:
df_cam_matrix = pd.read_csv(cal_path)

matrix_subjects = df_cam_matrix['subject'].unique().tolist()
matrix_subjects = sorted(matrix_subjects)
matrix_subjects = [int(subj.replace('S', '')) for subj in matrix_subjects]
map_matrix = {}
cur_num = 1
for i in range(1,29):
    if i in matrix_subjects:
        cur_num = i
    map_matrix[f'S{i}'] = f'S{cur_num}'

## Load camera matrices for specific subject
You can choose to load both stereo pairs or just one

In [ ]:
cal_path_pair1 = df_cam_matrix[(df_cam_matrix['main'] == main) & (df_cam_matrix['pair'] == pair1) & (df_cam_matrix['subject'] == map_matrix[subject])]['path'].values[0]
cal_path_pair2 = df_cam_matrix[(df_cam_matrix['main'] == main) & (df_cam_matrix['pair'] == pair2) & (df_cam_matrix['subject'] == map_matrix[subject])]['path'].values[0]

## Map subjects to wand annotations
Each time the Vicon system (marker-based motion capture system) was callibrated, the wand position was captured by all camera views and annotated. This map links all subjects to the last wand marker annotations captured.

In [ ]:
with open(wand_path, 'r') as f:
    markers = json.load(f)
keys = list(markers.keys())
subject_nums = [int(key.replace('S', '')) for key in keys if 'points' in markers[key]['C1']]
subject_nums = sorted(subject_nums)
map_nums = {}
cur_num = 1
for i in range(1,29):
    if i in subject_nums:
        cur_num = i
    map_nums[f'S{i}'] = f'S{cur_num}'

## Load and scale 3D WCS data
WCS is in mm, to it needs to be scaled by the callibration board's checker size.
- xyz: 3D coordinates of the human keypoints
- xyz_wand: 3D coordinates of the wand
- point_ids: list of keypoint ids

In [ ]:
with open(path_3d, 'r') as f:
    data_3d = json.load(f)

frame = data_3d[index]
xyz = frame['xyz']
xyz_wand = frame['xyz_wand']
point_ids = frame['point_ids']
points_skel = np.array(xyz)

wcs_skel = []
wcs_skel.append(points_skel)
wcs_skel = np.array(wcs_skel)
wcs_skel = wcs_skel.reshape(wcs_skel.shape[1], wcs_skel.shape[2])
wcs_skel[:, 0] = wcs_skel[:, 0]/cal_size
wcs_skel[:, 1] = wcs_skel[:, 1]/cal_size
wcs_skel[:, 2] = wcs_skel[:, 2]/cal_size

# wcs_skel = np.array(points_skel).reshape(len(points_skel), -1)
# wcs_skel[:, :3] /= cal_size

wcs_wand = []
wcs_wand.append(xyz_wand)
wcs_wand = np.array(wcs_wand)
wcs_wand = wcs_wand.reshape(wcs_wand.shape[1], wcs_wand.shape[2])
wcs_wand[:, 0] = wcs_wand[:, 0]/cal_size
wcs_wand[:, 1] = wcs_wand[:, 1]/cal_size
wcs_wand[:, 2] = wcs_wand[:, 2]/cal_size

## Load stereo calibration pairs camera matrices
1. Load data
2. Extract rotation matrix, R, and translation vector, t
3. Calculate each camera's projection matrix, P

Note: Origin/main camera will always be origin/0-values

In [ ]:
data_pair1 = np.load(cal_path_pair1)
data_pair2 = np.load(cal_path_pair2)

R1 = np.eye(3)  
T1 = np.array([0, 0, 0])  
R2 = data_pair1['R']
T2 = data_pair1['T']
T2 = np.array([T2[0][0], T2[1][0], T2[2][0]]).reshape((3,1))
R3 = data_pair2['R']
T3 = data_pair2['T']
T3 = np.array([T3[0][0], T3[1][0], T3[2][0]]).reshape((3,1))

R1_initial = R1.copy()
T1_initial = T1.reshape((3,1)).copy()
R2_initial = R2.copy()
T2_initial = T2.reshape((3,1)).copy()
R3_initial = R3.copy()
T3_initial = T3.reshape((3,1)).copy()

cameraMatrix1 = data_pair1['mtx_A']
cameraMatrix2 = data_pair1['mtx_B']
cameraMatrix3 = data_pair2['mtx_B']

RT1 = np.concatenate([R1, [[0],[0],[0]]], axis = -1)
P1 = cameraMatrix1 @ RT1
RT2 = np.concatenate([R2, T2], axis = -1)
P2 = cameraMatrix2 @ RT2
RT3 = np.concatenate([R3, T3], axis = -1)
P3 = cameraMatrix3 @ RT3

Ps = [P1, P2, P3]


## Load and project 2D wand to 3D
- The wand serves as an anchor between the WCS and the CCS
- The wand is the origin point and x-, y-, and z-directions of the WCS
- For each camera, the wand markers were manually annotated on the images
- You must project the wand from 2 or 3 camera views into the main/origin camera's 3D CCS

In [ ]:
all_cam_points = [
    markers[map_nums[subject]][f'C{main}']['points'],
    markers[map_nums[subject]][f'C{pair1}']['points'],
    markers[map_nums[subject]][f'C{pair2}']['points']
]

points = np.array(DLT_2_cameras(all_cam_points[0], all_cam_points[1], Ps[0], Ps[1]))
# OR
points = np.array(DLT_3_cameras(all_cam_points[0], all_cam_points[1], all_cam_points[2], Ps[0], Ps[1], Ps[2]))

## Calculate rotation and translation of WCS in CCS
- Calculate the x-, y-, and z-direction vectors of the WCS within the CCS
- Generate rotation matrix of WCS within CCS
- Set the translation vector of WCS within CCS as the middle point of the wand
- Generate camera matrux M used in projecting points in WCS to CCS

In [ ]:
x_direction = points[1] - points[0] 
z_direction = points[3] - points[1]  
x_direction /= np.linalg.norm(x_direction)
z_direction /= np.linalg.norm(z_direction)
y_direction = np.cross(x_direction, z_direction)

R_wcs = np.column_stack((x_direction, y_direction, z_direction))
M = np.column_stack((R_wcs, points[1]))
M = np.row_stack((M, np.array([0, 0, 0, 1])))
M_inv = np.linalg.inv(M)

## Project points from WCS to CCS

In [ ]:
c1_wand = np.array([projectInternal(M, p) for p in wcs_wand])
c1_skel = np.array([projectInternal(M, p) for p in wcs_skel])

## Correct projections
The WCS is linked to the wand, but the wand is a small ammount above the ground and needs to be aligned with the wand in the CCS

In [ ]:
order = [1, 0, 2, 3, 4] 
c1_wand = c1_wand[order, :]
diff = points - c1_wand
y_avg_wand = np.mean(c1_wand[:, 1])
y_avg_points = np.mean(points[:, 1])
y_diff = y_avg_points - y_avg_wand
x_avg_wand = np.mean(c1_wand[:, 0])
x_avg_points = np.mean(points[:, 0])
x_diff = x_avg_points - x_avg_wand
z_avg_wand = np.mean(c1_wand[:, 2])
z_avg_points = np.mean(points[:, 2])
z_diff = z_avg_points - z_avg_wand

c1_wand[:, 1] += y_diff
c1_skel[:, 1] += y_diff
c1_wand[:, 0] += x_diff
c1_skel[:, 0] += x_diff
c1_wand[:, 2] += z_diff
c1_skel[:, 2] += z_diff

## Project main/origin camera 3D points to 2D pixels

In [ ]:
Mint_1 = cameraMatrix1

points_c1_2d = np.array([projectPixel(Mint_1, p) for p in points])
c1_pixel_skel = np.array([projectPixel(Mint_1, p) for p in c1_skel])
c1_pixel_wand = np.array([projectPixel(Mint_1, p) for p in c1_wand])

## Project 3D main/origin -> 3D pair -> 2D pair pixel 
1. First project from main/origin camera CCS to the selected pair camera's CCS (both 3D)
2. Project 3D points in selected pair camera's CCS onto the 2D pixel plane

In [ ]:
R = R2_initial
t = T2_initial
Mint_2 = cameraMatrix2

points_internal_c2 = np.array([projectInternal(R, t, p) for p in points])
c2_3d_skel = np.array([projectInternal(R, t, p) for p in c1_skel])   
c2_3d_wand = np.array([projectInternal(R, t, p) for p in c1_wand])

points_pixel_c2 = np.array([projectPixel(Mint_2, p) for p in points_internal_c2])
c2_pixel_skel = np.array([projectPixel(Mint_2, p) for p in c2_3d_skel]) 
c2_pixel_wand = np.array([projectPixel(Mint_2, p) for p in c2_3d_wand])

## Project 3D main/origin -> 3D pair -> 2D pair pixel 
1. First project from main/origin camera CCS to the selected pair camera's CCS (both 3D)
2. Project 3D points in selected pair camera's CCS onto the 2D pixel plane

In [ ]:
R = R3_initial
t = T3_initial
Mint_3 = cameraMatrix3

points_internal_c3 = np.array([projectInternal(R, t, p) for p in points])
c3_3d_skel = np.array([projectInternal(R, t, p) for p in c1_skel])   
c3_3d_wand = np.array([projectInternal(R, t, p) for p in c1_wand])

points_pixel_c3 = np.array([projectPixel(Mint_3, p) for p in points_internal_c3])
c3_pixel_skel = np.array([projectPixel(Mint_3, p) for p in c3_3d_skel]) 
c3_pixel_wand = np.array([projectPixel(Mint_3, p) for p in c3_3d_wand])